# Split up data into single datasets

- create datasets per (set of) instruments for a specific experiments
- drop some samples based on quality criteria

In [ ]:
import logging

import pandas as pd
import matplotlib.pyplot as plt

from vaep.io import thermo_raw_files
from vaep.analyzers import analyzers

from config import erda_dumps
from config import defaults

import vaep
import vaep.io.filenames
from vaep.logging import setup_nb_logger

logger = setup_nb_logger()

FOLDER_DATA = defaults.FOLDER_DATA

In [ ]:
vaep.plotting.make_large_descriptors()
FIGSIZE=(15,10)

## Parameters

In [ ]:
DUMP: str = erda_dumps.FN_PROTEIN_GROUPS
# DUMP: str = erda_dumps.FN_PEPTIDES
# DUMP: str = erda_dumps.FN_EVIDENCE
FOLDER_DATASETS: str = f'single_datasets/{DUMP.stem}'
FILE_EXT = 'pkl'
SAMPLE_ID = 'Sample ID'

Make sure output folder exists

In [ ]:
FOLDER_DATASETS = defaults.FOLDER_DATA / FOLDER_DATASETS
FOLDER_DATASETS.mkdir(exist_ok=True, parents=True)
logger.info(f"Folder for datasets to be created: {FOLDER_DATASETS.absolute()}")

## Dumps

- load dumps
- load file to machine mappings

In [ ]:
data = pd.read_pickle(DUMP)
data = data.squeeze() # In case it is a DataFrame, not a series (-> leads to MultiIndex)
data

In [ ]:
logger.info(f"Number of rows (row = sample, feature, intensity): {len(data):,d}")

## Support per sample

In [ ]:
idx_non_sample = list(data.index.names)
idx_non_sample.remove(SAMPLE_ID)
idx_non_sample

In [ ]:
# M = data.index.droplevel(SAMPLE_ID).nunique() # very slow alternative, but 100% correct
M = vaep.io.filenames.read_M_features(DUMP.stem)
logger.info(f"Number of unqiue features: {M}")

In [ ]:
counts = data.groupby(SAMPLE_ID).count().squeeze()
counts.to_json(FOLDER_DATASETS / 'support_all.json', indent=4)
ax = (counts
      .sort_values()  # will raise an error with a DataFrame
      .reset_index(drop=True)
      .plot(rot=45,
            figsize=FIGSIZE,
            grid=True,
            xlabel='Count of samples ordered by number of features', 
            title=f'Support of {len(counts):,d} samples features over {M} features ({", ".join(idx_non_sample)})',
            ))
vaep.plotting.add_prop_as_second_yaxis(ax, M)
fig = ax.get_figure()
fig.tight_layout()
vaep.plotting.savefig(fig, name='support_all',
                      folder=FOLDER_DATASETS)


## Filter for odd samples

- fractionated samples
- GPF - Gas phase fractionation # Faims? DIA? 
- DIA
- CV

In [ ]:
# see 02_data_exploration_peptides

## Meta Data

- based on ThermoRawFileParser

In [ ]:
# sample_ids = data.index.levels[0] # assume first index position is Sample ID?
sample_ids = data.index.get_level_values(SAMPLE_ID).unique() # more explict
sample_ids

In [ ]:
df_meta = pd.read_csv('data/files_selected_metadata.csv', index_col=SAMPLE_ID)
date_col = 'Content Creation Date'
df_meta[date_col] = pd.to_datetime(df_meta[date_col])
df_meta = df_meta.loc[sample_ids]
df_meta

### Available instruments

In [ ]:
counts_instrument = df_meta.groupby(thermo_raw_files.cols_instrument)[date_col].agg(
    ['count', 'min', 'max']).sort_values(by=thermo_raw_files.cols_instrument[:2] + ['count'], ascending=False)
counts_instrument

In [ ]:
N_MIN_INSTRUMENT = 300
selected_instruments = counts_instrument.query(f"count >= {N_MIN_INSTRUMENT}")
fname = FOLDER_DATASETS / 'dataset_info'
selected_instruments.to_json(f"{fname}.json", indent=4)
selected_instruments.to_latex(f"{fname}.tex")
selected_instruments.to_excel(f"{fname}.xlsx")
logger.info(f"Save Information to: {fname} (as json, tex)")
selected_instruments

## Dump single experiments

In [ ]:
cols = selected_instruments.index.names

file_formats = {'pkl': 'to_pickle',
                'pickle': 'to_pickle',
                'csv': 'to_csv'}


for values in selected_instruments.index:
    mask = df_meta[cols] == values
    logger.info(f"Samples: {mask.all(axis=1).sum()}")
    sample_ids = df_meta.loc[mask.all(axis=1)].index
    dataset = data.loc[sample_ids]
    fname_dataset = vaep.io.get_fname_from_keys(values,
                                                folder=FOLDER_DATASETS,
                                                file_ext=f".{FILE_EXT}")

    logger.info(f'Dump dataset with N = {len(dataset)} to {fname_dataset}')
    _to_file_format = getattr(dataset, file_formats[FILE_EXT])
    _to_file_format(fname_dataset)

    fname_support = vaep.io.get_fname_from_keys(values,
                                            folder='.',
                                                file_ext="")
    fname_support = fname_support.stem + '_support'
    logger.info(f"Dump support to: {fname_support}")
    counts = dataset.groupby(SAMPLE_ID).count().squeeze()
    counts.to_json(FOLDER_DATASETS / f"{fname_support}.json", indent=4)
   
    M = dataset.index.droplevel(SAMPLE_ID).nunique() # very slow alternative, but 100% correct

    # plot:
    fig, ax = plt.subplots()
    ax = (counts
          .sort_values()  # will raise an error with a DataFrame
          .reset_index(drop=True)
          .plot(rot=45,
                ax=ax,
                figsize=FIGSIZE,
                grid=True,
                xlabel='Count of samples ordered by number of features',
                title=f'Support of {len(counts):,d} samples features over {M} features ({", ".join(idx_non_sample)})',
                ))
    vaep.plotting.add_prop_as_second_yaxis(ax, M)
    fig.tight_layout()
    vaep.plotting.savefig(fig, name=fname_support,
                          folder=FOLDER_DATASETS)


## Last example dumped

In [ ]:
dataset